# 설명

보라매 병원 홈페이지에서 각 진료과에 대해 모든 의사 정보와 각 의사 특화 진료 병명을 크롤링하는 코드입니다.
크롤링 결과는 'doctor_info.csv' 이름으로 저장됩니다.

# 코드 낱알

In [44]:
# 한 과 별로 여러명의 의사가 저장되어 있고, 사이트를 다르게 들어가서 각각의 과 별로 존재하는 모든 의사에 대해 정보를 저장하고자 함. 모든 과에 대해서. 총 34개의 과가 있으니 iteration 때 참고바람. 또한 각 과마다 의사의 수는 '의사 정보는 '~/div[2]/ul/li[2]/p[1]' 여기에서 ~/div[1~] 이 div 안의 숫자로 구분함.' 이걸 참고해서, 과마다 총 몇 명의 의사가 있는지를 바탕으로 모든 의사에 대해 크롤링해야함.
# 과를 돌아다니는 방법은, 밑에 '첫번째 과의 의료진 정보 탭으로 넘어가기' 이 부분 참고하셈.
# 다음 과로 접속하는 모듈도 만들어야함
# 
# 저장할 데이터 df의 칼럼은 각각의 의사별로, dept_name, doc_name, special_disease 이렇게 세개임
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)
URL = 'https://www.brmh.org/medical/medi_doctor_info.do?site=001&user_code=010012000&mcode=001002000'
driver.get(URL)
time.sleep(3)
# 첫번째 과의 의료진 정보 새창을 새롭게 여는거임 ;; 두번째 과의 의료진 정보 탭 주소는 '/html/body/div[2]/div/div[3]/div/div[2]/div[3]/ul/li[34]/div/ul/li[2]/a' 이거니까, 과끼리 iteration할 떄 참고 바람.
# tab_element = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[3]/div/div[2]/div[3]/ul/li[1]/div/ul/li[2]/a')
# tab_element.click()

# 첫번째 과의 의료진 정보 탭 안에서, 과 이름 구하기
dept_name=driver.find_element(By.XPATH,'/html/body/div[2]/div/div[3]/div/div[1]/h3').text.strip()
print(dept_name)
time.sleep(1)
# 첫번째 의사의 정보 추출, 의사 정보는 '~/div[2]/ul/li[2]/p[1]' 여기에서 ~/div[1~] 이 div 안의 숫자로 구분함.
# 예를 들어 첫번째 과의 마지막 의사 정보는 이 안에 있음. '/html/body/div[2]/div/div[3]/div/div[4]/div/div[5]/ul/li[2]/p[1]'
# 그리고 추출 형태 예시는 이런 형태로 나옴(두 줄의 텍스트 형태라서, 이름만 크롤링해서 doc_name으로 저장하기
#'가정의학과
#오범조 자세히 보기'
doc_name=driver.find_element(By.XPATH,'/html/body/div[2]/div/div[3]/div/div[4]/div/div[1]/ul/li[2]/p[1]').text.strip()
print(doc_name)
#/html/body/div[2]/div/div[3]/div/div[4]/div/div[1]/ul/li[2]/p[1]/text()
# # 첫번째 과의 첫번째 의사의 special_disease 추출하면 '만성피로, 체중감소, 스트레스, 금연' 이런 형태의 텍스트로 나옴. 이걸 쉼표랑 blank 떼고, 각 의사마다 리스트로 정리바람
# special_disease=driver.find_element(By.XPATH,'/html/body/div[2]/div/div[3]/div/div[4]/div/div[1]/ul/li[2]/p[2]').text.strip()
# print(special_disease)




가정의학과
가정의학과
김종성 자세히 보기


# 모듈화

In [ ]:
#임포트
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep
import random
import re

from selenium import webdriver
import sys

In [45]:
def extract_doctor_info(driver):
    """
    현재 부서의 의료진정보 페이지에서 의사 정보를 추출합니다.
    - 부서 이름: /html/body/div[2]/div/div[3]/div/div[1]/h3
    - 각 의사의 정보는 '/html/body/div[2]/div/div[3]/div/div[4]/div' 내 각 자식 div에 있으며,
      의사 이름은 각 자식 div 내의 './ul/li[2]/p[1]/span'에서 두 번째 줄의 첫 단어를 추출합니다.
      전문 질환 정보는 './ul/li[2]/p[2]'에서 추출한 후 쉼표로 분리합니다.
    반환값: 각 의사에 대해 딕셔너리 ({'dept_name', 'doc_name', 'special_disease': [리스트]})
    """
    dept_name = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[3]/div/div[1]/h3').text.strip()
    doctor_container = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[3]/div/div[4]/div')
    doctor_divs = doctor_container.find_elements(By.XPATH, './div')
    doctors = []
    for div in doctor_divs:
        full_text = div.find_element(By.XPATH, './ul/li[2]/p[1]').text.strip()
        lines = full_text.splitlines()
        if len(lines) >= 2:
            doc_name = lines[1].split()[0]
        else:
            doc_name = full_text
        special_text = div.find_element(By.XPATH, './ul/li[2]/p[2]').text.strip()
        special_list = [s.strip() for s in special_text.split(',')]
        doctors.append({
            'dept_name': dept_name,
            'doc_name': doc_name,
            'special_disease': special_list
        })
    return doctors


def main():
    total_depts = 34  # 총 부서(과) 수
    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument('window-size=1380,900')
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(5)
    
    # 부서 탭이 있는 메인 페이지 URL (실제 URL로 변경)
    URL = 'https://www.brmh.org/custom/medi_team.do?site=001&user_code=010012000'
    driver.get(URL)
    sleep(3)
    
    all_doctors = []  # 모든 의사 정보를 저장할 리스트
    
    for i in range(1, total_depts + 1):
        # 부서 탭의 절대 XPath (예: 1번째 부서: li[1], 2번째 부서: li[2], ...)
        xpath = f"/html/body/div[2]/div/div[3]/div/div[2]/div[3]/ul/li[{i}]/div/ul/li[2]/a"
        dept_tab = driver.find_element(By.XPATH, xpath)
        url = dept_tab.get_attribute("href")
        print(f"부서 {i}의 URL: {url}")
        
        # 해당 부서 페이지로 접속
        driver.get(url)
        sleep(2)  # 페이지 로딩 대기
        
        # 의료진 정보를 추출
        doctors = extract_doctor_info(driver)
        print(f"부서 {i}의 의료진 정보:")
        for doc in doctors:
            print(doc)
        all_doctors.extend(doctors)
        
        # 메인 부서 탭 페이지로 돌아가기
        driver.back()
        sleep(2)
    
    driver.quit()
    
    df = pd.DataFrame(all_doctors)
    df.to_csv("doctor_info.csv", index=False, encoding="utf-8-sig")
    print("모든 의사 정보 크롤링 완료! doctor_info.csv 파일 저장됨.")

if __name__ == "__main__":
    main()

부서 1의 URL: https://www.brmh.org/medical/medi_doctor_info.do?site=001&user_code=010012000&mcode=001002000
부서 1의 의료진 정보:
{'dept_name': '가정의학과', 'doc_name': '김종성', 'special_disease': ['만성피로', '체중감소', '스트레스', '금연']}
{'dept_name': '가정의학과', 'doc_name': '오범조', 'special_disease': ['만성피로', '비만', '스트레스', '금연']}
{'dept_name': '가정의학과', 'doc_name': '한수정', 'special_disease': ['건강증진', '건강검진', '만성질환관리(고혈압/당뇨병/고지혈증/골다공증 등)', '암경험자 건강관리', '비만', '금연', '예방접종']}
{'dept_name': '가정의학과', 'doc_name': '전혜령', 'special_disease': ['만성질환관리(고혈압/당뇨병/고지혈증/골다공증 등)', '암경험자 건강관리', '비만', '금연', '예방접종']}
{'dept_name': '가정의학과', 'doc_name': '임상강사', 'special_disease': ['만성질환', '금연', '건강검진']}
부서 2의 URL: https://www.brmh.org/medical/medi_doctor_info.do?site=001&user_code=010012000&mcode=001003000
부서 2의 의료진 정보:
{'dept_name': '감염내과', 'doc_name': '박상원', 'special_disease': ['결핵', '발열 질환', '예방접종', '대상포진', '폐렴', '독감', '상처감염', '방광염/신우신염', '감염성 설사병', '편도선염', '성병', '면역저하환자']}
{'dept_name': '감염내과', 'doc_name': '오홍상', 'special_disease': ['